In [1]:
import pandas as pd
import numpy as np
import glob
import pyarrow as pa
import pyarrow.parquet as pq
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df_all = pl.read_parquet('./data/2022_data_selected.parquet')

In [3]:
df_all.head()

date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_12_normalized,smart_12_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw
str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""2022-03-11""","""ZLW18P9K""","""ST14000NM001G""",14000519643136,0,82,145176856,90,0,100,12,100,0,89,788618680,87,11590,100,0,100,12,100,0,100,0,100,2,99,2710,33,33,100,0,100,0,200,0
"""2022-03-11""","""ZLW0EGC7""","""ST12000NM001G""",12000138625024,0,82,151519352,99,0,100,1,100,0,83,178873441,84,14836,100,0,100,1,100,0,100,0,100,0,99,3909,48,48,100,0,100,0,200,0
"""2022-03-11""","""ZA1FLE1P""","""ST8000NM0055""",8001563222016,0,77,49979496,91,0,100,7,100,0,93,2115550504,84,14448,100,0,100,7,100,0,100,0,100,121,100,764,38,38,100,0,100,0,200,0
"""2022-03-11""","""ZA16NQJR""","""ST8000NM0055""",8001563222016,0,74,27873200,89,0,100,10,100,0,94,2634135447,52,42621,100,0,100,10,100,0,100,0,100,16,68,64822,34,34,100,0,100,0,200,0
"""2022-03-11""","""1050A084F97G""","""TOSHIBA MG07AC...",14000519643136,0,100,0,100,7770,100,3,100,0,100,0,81,7758,100,0,100,3,null,null,null,null,100,1,100,13,100,26,100,0,100,0,200,0


In [4]:
df_all['serial_number'].value_counts()

serial_number,counts
i64,u32
202328795208,364
220335800584,94
220335800344,94
220335800351,78
null,80354909
220335800598,93
214220802950,94
202328782774,364
220335800342,93


In [ ]:
df_all['model'].value_counts()

In [ ]:
df_all

In [ ]:
df_all['failure'].value_counts()

In [ ]:
df_all.null_count().transpose()

In [ ]:
def get_nan_count_percent(df, divisor=None):
    """Calculates the number of nan values per column,
        both as an absolute amount and as a percentage of some pre-defined "total" amount
        
        WARNING: Return value is of the same type (pd.DataFrame or dask.dataframe) as the
        input. It is up to the caller to handle this accordingly.
    
    Arguments:
        df {polars.DataFrame/dask.dataframe} -- dataframe whose nan count to generate
    
    Keyword Arguments:
        divisor {int/float} -- the "total" amount for calculating percentage. 
                                If value in count column is n, value in percent column
                                will be n/divisor.
                                If not provided, number of rows is used by default
                                (default: {None})
    
    Returns:
        ret_df {pandas.DataFrame/dask.dataframe} -- dataframe with counts and percentages
                                                    of nans in each column of input df.
                                                    Column name is the index, "count" and
                                                    "percent" are the two columns.
    """
    # if total count is not provided, use the number of rows
    if divisor is None:
        # NOTE: len must be used, not shape because in case of dask dataframe
        # shape returns a delayed computation, not an actual value. but
        # len returns an actual value
        divisor = len(df)

    # get count and convert series to dataframe
    ret_df = df.null_count().transpose()
    ret_df = ret_df.with_column(pl.lit(ret_df["column_0"] / divisor).alias('percent'))

    # add percent column
    #ret_df["percent"] = ret_df["column_0"] / divisor

    return ret_df

In [ ]:
get_nan_count_percent(df_all).sort(by='percent')

In [ ]:
df_Seagate = df_all.filter(pl.col("model") == "ST4000DM000")

In [ ]:
df_Seagate['failure'].value_counts()

In [ ]:
df = df_Seagate.to_pandas()
df.head()

In [ ]:
corr = df.corr()

In [ ]:
df_failed = df[df['failure'] == 1]

In [ ]:
sns.histplot(data=df, x='failure')

In [ ]:
df_filtered = df[(df['serial_number'] == "Z305B8PX")]

In [ ]:
sns.lineplot(data=df_filtered, x="date", y='capacity_bytes')

In [ ]:
sns.histplot(data=df_Seagate, x='serial_number')
sns.set(rc={'figure.figsize':(11.7,12.27)})
plt.yticks();

In [ ]:
df_Seagate.null_count()

In [ ]:
df_all = df_all.with_columns(pl.col('date').str.strptime(pl.Date, fmt='%Y-%m-%d'), strict=False)

In [ ]:
df_failure = df_all.filter(pl.col("failure") == 1)

In [ ]:
df_failure['model'].value_counts().sort(by='counts', descending=True)

In [ ]:
sns.histplot(data=df_failure, y='model')
sns.set(rc={'figure.figsize':(11.7,12.27)})
plt.yticks();

In [ ]:
df_failed_example = df_all.filter(pl.col("serial_number") == "ZJV3BYAY")

In [ ]:
df_sorted = df_failed_example.sort(by='date')

In [ ]:
df_sorted

In [ ]:
fig, ax = plt.subplots(32, figsize=(20,40))
for i in range(32):
    graph1 = sns.lineplot(data=df_sorted, x='date', y=df_sorted.columns[i+5], ax=ax[i])
    

In [ ]:
# # Plot views, likes, dislikes and comments per category
# fig, ax = plt.subplots(2,3, figsize=(20,15)) # create subplots on 2 rows and 2 columns
# plt.suptitle('sqft_living, bathrooms, sqft_lot and price per grades', fontsize=20) 
# fig.tight_layout()   # tight_layout automatically adjusts subplot params so that the subplot(s) fits in to the figure area
# plt.subplots_adjust(hspace = .6, wspace = .2, top = .9) # adjusts the space between the single subplots

# f1 = sns.barplot(data = df_sorted, x="date", y = df_Jakob1['sqft_living'], ax=ax[0,0]) # creates barchart for number of views
# f1.tick_params(axis='x', labelrotation=90) # sets the label y-axis
# f1.set(xlabel=None) # sets the x-axis label
# f1.set_ylabel("sqft_living distribution", fontsize = 10) # limits the x-axis values from 0 to 1000000
# f1.set_title("sqft_living per grades", fontsize = 15);  # sets title for subplot

# f2= sns.barplot(x=df_Jakob1["grades"], y = df_Jakob1['bathrooms'], ax=ax[0,1])
# f2.tick_params(axis='x', labelrotation=90)
# f2.set(xlabel=None)
# f2.set_ylabel("bathrooms distribution", fontsize = 10)
# f2.set_title("bathrooms for housing per grades", fontsize = 15);

# f3 = sns.barplot(x=df_Jakob1["grades"], y = df_Jakob1['sqft_lot'], ax=ax[0,2])
# f3.tick_params(axis='x', labelrotation=90)
# f3.set(xlabel=None)
# f3.set_ylabel("sqft_lot distribution", fontsize = 10)
# f3.set_title("sqft_lot for housing per grades", fontsize = 15);

# f4 = sns.barplot(x=df_Jakob1["grades"], y = df_Jakob1['bedrooms'], ax=ax[1,0])
# f4.tick_params(axis='x', labelrotation=90)
# f4.set(xlabel=None)
# f4.set_ylabel("bedrooms distribution", fontsize = 10)
# f4.set_title("bedrooms for housing per grades", fontsize = 15);

# f5 = sns.barplot(x=df_Jakob1["grades"], y = df_Jakob1['floors'], ax=ax[1,1])
# f5.tick_params(axis='x', labelrotation=90)
# f5.set(xlabel=None)
# f5.set_ylabel("distribution of floors", fontsize = 10)
# f5.set_title("floors for housing per grades", fontsize = 15);

# fig.delaxes(ax[1][2]);